# Scipion Bridge Demo

In [4]:
import os
from pathlib import Path
import numpy as np

import scipion_bridge
import scipion_bridge.ffi.scipion as scipion
from scipion_bridge.core.typed import common, proxy
# from scipion_bridge.ffi.blocres import blocres
# from scipion_bridge.proxy import OutputInfo

import shutil

from utils import download as D
scipion_bridge.core.environment.configure_default_env()

import matplotlib.pyplot as plt

### Download Data

In [5]:
DOWNLOAD_PATH = Path("data/downloaded")
EMDB_ENTRY = 41510

In [6]:
metadata = D.download_emdb_metadata(entry_id=EMDB_ENTRY)

os.makedirs(DOWNLOAD_PATH, exist_ok=True)

emdb_map = D.download_emdb_map(EMDB_ENTRY, DOWNLOAD_PATH)
map_1, map_2 = D.download_halfmaps(EMDB_ENTRY, DOWNLOAD_PATH)
pdb_model = D.download_pdb_model("8tqo", DOWNLOAD_PATH) # metadata.pdb_id

metadata

EMDBMetadata(pdb_id='7l70', resolution=3.1, sampling=0.835, size=256, org_x=0, org_y=0, org_z=0)

### Create Volume from PDB

In [8]:
volume = scipion.xmipp_volume_from_pdb(
    pdb_model,
    center_pdb="-v 0",
    sampling=metadata.sampling,
    size=metadata.size,
).typed(astype=scipion_bridge.core.typed.volume.SpiderFile, copy_data=False)

# volume = scipion.xmipp_volume_align(
#     embdb_map=emdb_map,
#     volume=volume,
#     local=True,
# )

volume

[XMIPP] scipion run xmipp_volume_from_pdb -i data/downloaded/pdb8tqo.ent -o /tmp/tmp2puw8rqj --centerPDB -v 0 --sampling 0.835 --size 256
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [9]:
# [XMIPP] scipion run xmipp_volume_align --i1 data/downloaded/emd_41510.map --i2 /tmp/tmpvmzz70hb --local --apply /tmp/tmp09zqxngg.vol

### Mask from Volume

In [10]:
import logging


mask = scipion.xmipp_transform_threshold(
    volume, select="below 0.02", substitute="binarize"
)

mask

[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmp2puw8rqj.vol -o /tmp/tmpxarl4c9g.vol --select below 0.02 --substitute binarize
Input File: /tmp/tmp2puw8rqj.vol
Output File: /tmp/tmpxarl4c9g.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [11]:
print(f"Number of non-zero elements: {np.count_nonzero(mask)}")

Number of non-zero elements: 487083


In [12]:
from ipywidgets import interact, IntSlider
@interact(i=IntSlider(min=0, max=255))
def plot_slices(i=125):
    fig, (ax_1, ax_2) = plt.subplots(1, 2)
    ax_1.imshow(np.array(mask)[i], cmap="gray")
    ax_2.imshow(np.array(volume)[i], cmap="gray")

interactive(children=(IntSlider(value=0, description='i', max=255), Output()), _dom_classes=('widget-interact'…

In [13]:
shape = (256, 256, 256)
z, y, x = np.indices(shape)

center = np.array(shape) / 2
radius = 100

dist_squared = (x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2
sphere_mask = dist_squared <= radius**2

sphere_mask = scipion.xmipp_transform_threshold(
    sphere_mask, select="below 0.5", substitute="binarize"
)

sphere_mask

/home/max/Documents/scipion-bridge/src/scipion_bridge/core/typed/resolve.py:241: UserWarning: Attempted register a resolver for existing transform 'ndarray' -> 'SpiderFile' ('resolve_output_to_proxy' vs 'resolve_output_to_proxy')
  warnings.warn(


[XMIPP] scipion run xmipp_transform_threshold -i /tmp/tmpy2jhuv7s.vol -o /tmp/tmpcdj12_jj.vol --select below 0.5 --substitute binarize
Input File: /tmp/tmpy2jhuv7s.vol
Output File: /tmp/tmpcdj12_jj.vol
Xmipp command detected
Scipion v3.7.1 - Eugenius


Output()

In [14]:
np.count_nonzero(sphere_mask)

4187857